In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='../Analysis/human_rating_bycat'

In [3]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

(Test) Use quantiles to remove outliers 

In [4]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [5]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [7]:
vgg_df=pd.read_csv('vgg_scores_wordnet_id_closest_label.csv',index_col=0)
vgg_df['typicality_score'] = np.round(vgg_df['closest_prob'] * 100,2)
vgg_df['7_score']= np.round(vgg_df['typicality_score']*7/100,2)
vgg_df['type'] = vgg_df['type'].apply(lambda x: x.lower())

In [8]:
vgg_df['number'] = vgg_df['dir'].apply(lambda x: int(x.split('\\')[-1].split('.')[0]))
vgg_df = vgg_df.sort_values(['type','number']).reset_index(drop=True)

# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [16]:
final_corr = dict()
for t in np.unique(vgg_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings =vgg_df[vgg_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
#     net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.19573220898699903
beach:  -0.19999999999999998
car:  -0.15882352941176472
church:  0.07064019572463123
lighthouse:  0.5529411764705882
mountain:  0.5342164801675237
mug:  -0.04274135279917027
plane:  0.469462967419945


In [13]:
vgg_df

,type,dir,id_labels,readable_labels,similarity_score,closest_id_label,closest_similarity_score,closest_readable_label,closest_prob,typicality_score,7_score,number
0,banana,../Image/All_Cropped\Banana\1.jpg,"{'07753592-n': 0.7968792915344238, '07716358-n...","{'banana': 0.7968792915344238, 'zucchini': 0.0...","{'07753592-n': 1.0, '07716358-n': 0.1428571428...",07753592-n,1.000000,banana,0.796879,79.69,5.58,1
1,banana,../Image/All_Cropped\Banana\2.jpg,"{'01924916-n': 0.27627280354499817, '03532672-...","{'flatworm': 0.27627280354499817, 'hook': 0.22...","{'01924916-n': 0.07142857142857142, '03532672-...",07753592-n,1.000000,banana,0.012688,1.27,0.09,2
2,banana,../Image/All_Cropped\Banana\3.jpg,"{'07753592-n': 0.4913323223590851, '03065424-n...","{'banana': 0.4913323223590851, 'coil': 0.10712...","{'07753592-n': 1.0, '03065424-n': 0.0909090909...",07753592-n,1.000000,banana,0.491332,49.13,3.44,3
3,banana,../Image/All_Cropped\Banana\4.jpg,"{'07753592-n': 0.24134351313114166, '07720875-...","{'banana': 0.24134351313114166, 'bell pepper':...","{'07753592-n': 1.0, '07720875-n': 0.125, '0439...",07753592-n,1.000000,banana,0.241344,24.13,1.69,4
4,banana,../Image/All_Cropped\Banana\5.jpg,"{'07753592-n': 0.9275352954864502, '07718747-n...","{'banana': 0.9275352954864502, 'artichoke': 0....","{'07753592-n': 1.0, '07718747-n': 0.2, '037869...",07753592-n,1.000000,banana,0.927535,92.75,6.49,5
...,...,...,...,...,...,...,...,...,...,...,...,...
123,plane,../Image/All_Cropped\Plane\12.jpg,"{'02690373-n': 0.2403966337442398, '04592741-n...","{'airliner': 0.2403966337442398, 'wing': 0.217...","{'02690373-n': 0.5, '04592741-n': 0.1, '034968...",02690373-n,0.500000,airliner,0.240397,24.04,1.68,12
124,plane,../Image/All_Cropped\Plane\13.jpg,"{'04037443-n': 0.15428905189037323, '04285008-...","{'racer': 0.15428905189037323, 'sports car': 0...","{'04037443-n': 0.1, '04285008-n': 0.1, '026903...",02690373-n,0.500000,airliner,0.140950,14.09,0.99,13
125,plane,../Image/All_Cropped\Plane\14.jpg,"{'04044716-n': 0.349433034658432, '04258138-n'...","{'radio telescope': 0.349433034658432, 'solar ...","{'04044716-n': 0.07142857142857142, '04258138-...",04044716-n,0.071429,"radio telescope, radio reflector",0.349433,34.94,2.45,14
126,plane,../Image/All_Cropped\Plane\15.jpg,"{'04273569-n': 0.7837613821029663, '02951358-n...","{'speedboat': 0.7837613821029663, 'canoe': 0.0...","{'04273569-n': 0.125, '02951358-n': 0.125, '02...",04273569-n,0.125000,speedboat,0.783761,78.38,5.49,15


In [14]:
for t in

,banana_1,banana_2,banana_3,banana_4,banana_5,banana_6,banana_7,banana_8,banana_9,banana_10,...,plane_7,plane_8,plane_9,plane_10,plane_11,plane_12,plane_13,plane_14,plane_15,plane_16
0,7,5,6,5,5,7,7,6,6,4,...,7,3,7,7,2,5,6,4,4,7
1,7,6,7,6,6,6,6,6,6,6,...,7,3,4,7,3,6,5,5,3,7
2,7,7,7,6,6,7,7,7,7,7,...,7,4,6,7,4,5,6,5,5,7
3,7,7,7,2,3,7,7,5,5,4,...,7,2,4,7,2,3,3,1,2,7
4,7,6,7,5,5,7,7,6,6,5,...,6,4,5,6,4,4,5,3,4,7
5,5,3,3,5,6,7,4,7,5,6,...,7,2,5,7,4,5,6,4,3,6
6,7,7,7,7,7,7,7,7,7,7,...,7,2,7,7,2,4,4,7,5,7
7,6,6,5,1,2,6,4,4,7,2,...,6,4,4,5,3,3,5,1,3,5
8,5,4,4,2,2,6,5,6,5,3,...,4,6,4,5,6,3,6,2,5,3
9,7,7,7,6,6,7,7,7,7,6,...,7,4,5,7,3,5,5,2,3,7


In [10]:
print(np.mean(list(final_corr.values())))

0.17767851831984402


after dropping outliers

In [11]:
final_corr = dict()
for t in np.unique(vgg_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = vgg_df[vgg_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.16924213559026233
beach:  -0.23416814791772844
car:  -0.1723124107319134
church:  0.06916852498036807
lighthouse:  0.5567016346723356
mountain:  0.44775395197848766
mug:  0.029520367555897542
plane:  0.3716830330568026


In [12]:
print(np.mean(list(final_corr.values())))

0.15469863614806398
